In [87]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [88]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.2.3
3.2.1


In [89]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [90]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
"""
    Parameters
    ----------
    problem_formulation_id : int {0, ..., 5}
                             problem formulations differ with respect to the objectives
                             0: Total cost, and casualties
                             1: Expected damages, costs, and casualties
                             2: expected damages, dike investment costs, rfr costs, evacuation cost, and casualties
                             3: costs and casualties disaggregated over dike rings, and room for the river and evacuation costs
                             4: Expected damages, dike investment cost and casualties disaggregated over dike rings and room for the river and evacuation costs
                             5: disaggregate over time and space

    Notes
    -----
    problem formulations 4 and 5 rely on ArrayOutcomes and thus cannot straightforwardly
    be used in optimizations

    """
problem_formulation = 3
dike_model, planning_steps = get_model_for_problem_formulation(problem_formulation)

In [91]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [92]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [93]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage', 'A.1_Dike Investment Costs'), function=<function sum_over at 0x000001B501698310>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths',), function=<function sum_over at 0x000001B501698310>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage', 'A.2_Dike Investment Costs'), function=<function sum_over at 0x000001B501698310>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths',), function=<function sum_over at 0x000001B501698310>)
ScalarOutcome('A.3 Total Costs', variable_name=('A.3_Expected Annual Damage', 'A.3_Dike Investment Costs'), function=<function sum_over at 0x000001B501698310>)
ScalarOutcome('A.3_Expected Number of Deaths', variable_name=('A.3_Expected Number of Deaths',), function=<function sum_over at 0x000001B501698310>)
ScalarOutcome('A.4 Total Costs', variable_name=('A.4_Expected Annual Dama

In [94]:
# running the model through EMA workbench
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=2, policies=4)

[MainProcess/INFO] performing 2 scenarios * 4 policies * 1 model(s) = 8 experiments
100%|████████████████████████████████████████████| 8/8 [00:09<00:00,  1.13s/it]
[MainProcess/INFO] experiments finished


In [95]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,A.4_Bmax,A.4_Brate,A.4_pfail,A.5_Bmax,A.5_Brate,A.5_pfail,discount rate 0,discount rate 1,discount rate 2,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,3_RfR 1,3_RfR 2,4_RfR 0,4_RfR 1,4_RfR 2,A.1_DikeIncrease 0,A.1_DikeIncrease 1,A.1_DikeIncrease 2,A.2_DikeIncrease 0,A.2_DikeIncrease 1,A.2_DikeIncrease 2,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,61,219.653491,1.5,0.361758,238.282430,1.0,0.346764,280.431634,1.0,0.082714,65.160714,10.0,0.383494,299.089103,1.5,0.588434,4.5,1.5,3.5,0,1,0,0,0,0,0,0,0,1,1,1,1,1,0,2,8,0,9,6,4,9,8,6,9,3,2,3,7,10,2,258,254,dikesnet
1,114,151.321371,1.0,0.605159,98.902875,10.0,0.829465,115.054205,1.5,0.525065,196.445990,1.0,0.874298,172.191746,10.0,0.430157,1.5,3.5,1.5,0,1,0,0,0,0,0,0,0,1,1,1,1,1,0,2,8,0,9,6,4,9,8,6,9,3,2,3,7,10,2,259,254,dikesnet
2,61,219.653491,1.5,0.361758,238.282430,1.0,0.346764,280.431634,1.0,0.082714,65.160714,10.0,0.383494,299.089103,1.5,0.588434,4.5,1.5,3.5,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,8,1,7,7,5,5,6,0,0,1,0,8,1,1,4,0,258,255,dikesnet
3,114,151.321371,1.0,0.605159,98.902875,10.0,0.829465,115.054205,1.5,0.525065,196.445990,1.0,0.874298,172.191746,10.0,0.430157,1.5,3.5,1.5,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,8,1,7,7,5,5,6,0,0,1,0,8,1,1,4,0,259,255,dikesnet
4,61,219.653491,1.5,0.361758,238.282430,1.0,0.346764,280.431634,1.0,0.082714,65.160714,10.0,0.383494,299.089103,1.5,0.588434,4.5,1.5,3.5,1,1,0,1,0,1,1,1,1,0,0,0,1,0,1,9,6,5,2,2,9,0,6,3,8,10,10,7,9,0,3,258,256,dikesnet
5,114,151.321371,1.0,0.605159,98.902875,10.0,0.829465,115.054205,1.5,0.525065,196.445990,1.0,0.874298,172.191746,10.0,0.430157,1.5,3.5,1.5,1,1,0,1,0,1,1,1,1,0,0,0,1,0,1,9,6,5,2,2,9,0,6,3,8,10,10,7,9,0,3,259,256,dikesnet
6,61,219.653491,1.5,0.361758,238.282430,1.0,0.346764,280.431634,1.0,0.082714,65.160714,10.0,0.383494,299.089103,1.5,0.588434,4.5,1.5,3.5,0,0,1,1,1,0,1,1,1,1,1,1,0,1,1,1,4,8,3,8,2,3,4,10,4,8,4,10,3,6,2,258,257,dikesnet
7,114,151.321371,1.0,0.605159,98.902875,10.0,0.829465,115.054205,1.5,0.525065,196.445990,1.0,0.874298,172.191746,10.0,0.430157,1.5,3.5,1.5,0,0,1,1,1,0,1,1,1,1,1,1,0,1,1,1,4,8,3,8,2,3,4,10,4,8,4,10,3,6,2,259,257,dikesnet


In [96]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,1.188578e+08,0.0,2.676838e+08,0.000000,1.466476e+08,0.000000,4.055564e+07,0.000000,1.606408e+08,0.000000,9.604000e+08,0.000000
1,1.188578e+08,0.0,2.676838e+08,0.000000,1.466476e+08,0.000000,4.055564e+07,0.000000,1.606408e+08,0.000000,9.604000e+08,0.000000
2,2.112759e+08,0.0,2.373924e+08,0.000000,7.269766e+07,0.080506,3.129457e+07,0.004330,9.451434e+07,0.010822,6.048000e+08,0.000000
3,2.112759e+08,0.0,2.373924e+08,0.000000,3.139880e+07,0.000000,2.344263e+07,0.000255,1.622690e+08,0.059445,6.048000e+08,0.000000
4,2.567634e+08,0.0,1.931185e+08,0.000268,1.078124e+08,0.013597,8.010105e+07,0.000000,1.113913e+08,0.000000,1.209100e+09,3685.848833
5,2.567634e+08,0.0,1.915505e+08,0.000000,6.259007e+07,0.000253,8.010105e+07,0.000000,1.113913e+08,0.000000,1.209100e+09,53.692097
6,1.724348e+08,0.0,1.997049e+08,0.000000,1.093408e+08,0.000000,4.263127e+07,0.000000,1.629821e+08,0.000000,1.488100e+09,0.000000
7,1.724348e+08,0.0,1.997049e+08,0.000000,1.093408e+08,0.000000,4.263127e+07,0.000000,1.629821e+08,0.000000,1.488100e+09,0.000000


In [97]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations
# policy 4 doe niks
'''
A1 = Doesburg Upstream
A2 = Cortenoever Upmidstream
A3 = Zutphen Midstream
A4 = Gorssel Downmidstream
A5 = Deventer Downstream
0_RFR = Project Olburgen
1_RFR = Project Havikerwaard
2_RFR = project Tichelbeekse
3_RFR = Project Welsummer
4_RFR = Obstakelsverwijderen
dan 0 of 1 of 2 is de timestep waarin het wordt geddaan
'''



def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "2_RfR 0": 1, "2_RfR 1": 1, "2_RfR 2": 1, "A.3_DikeIncrease 0": 5, "A.1_DikeIncrease 0": 5}
        )
    ),
        Policy(
        "policy x",
        **dict(
            get_do_nothing_dict(),
            **{"3_RfR 0": 1, "3_RfR 1": 1, "3_RfR 2": 1, "4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1}
        )
    ),
    
    # Policy(
    #     "policy 2",
    #     **dict(
    #         get_do_nothing_dict(),
    #         **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
    #     )
    # ),
    # Policy(
    #     "policy 3",
    #     **dict(
    #         get_do_nothing_dict(),
    #         **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
    #     )
    # ),
    # Policy(
    #     "policy 4",
    #     **dict(
    #         get_do_nothing_dict(),
            
    #     )
    # ),
]

In [98]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 20
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] performing 20 scenarios * 2 policies * 1 model(s) = 40 experiments
100%|██████████████████████████████████████████| 40/40 [00:46<00:00,  1.17s/it]
[MainProcess/INFO] experiments finished


In [99]:
experiments, outcomes = results

In [100]:
# only works because we have scalar outcomes
outcomes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
outcomes=pd.DataFrame(outcomes)
outcomes

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,5.397251e+07,0.000000,2.568006e+07,0.020833,2.879840e+07,0.000000,1.842313e+08,0.067260,0.000000e+00,0.000000,3.459000e+08,0.0
1,5.397251e+07,0.000000,1.865696e+07,0.021061,2.879840e+07,0.000000,1.907112e+07,0.011724,0.000000e+00,0.000000,3.459000e+08,0.0
2,5.397251e+07,0.000000,7.020360e+06,0.006593,2.879840e+07,0.000000,0.000000e+00,0.000000,5.341134e+07,0.048778,3.459000e+08,0.0
3,5.397251e+07,0.000000,1.751092e+08,0.182915,2.879840e+07,0.000000,2.780590e+07,0.014913,0.000000e+00,0.000000,3.459000e+08,0.0
4,5.397251e+07,0.000000,8.652724e+06,0.006745,4.484990e+07,0.022996,1.632386e+07,0.007215,0.000000e+00,0.000000,3.459000e+08,0.0
5,5.397251e+07,0.000000,2.113737e+07,0.020747,2.879840e+07,0.000000,2.097255e+07,0.010986,3.978490e+08,0.368189,3.459000e+08,0.0
6,5.397251e+07,0.000000,0.000000e+00,0.000000,2.879840e+07,0.000000,1.409782e+07,0.008085,1.870280e+08,0.179877,3.459000e+08,0.0
7,5.397251e+07,0.000000,7.055213e+08,0.547362,2.879840e+07,0.000000,0.000000e+00,0.000000,2.839959e+07,0.023241,3.459000e+08,0.0
8,5.397251e+07,0.000000,2.808794e+08,0.286851,2.879840e+07,0.000000,3.855804e+07,0.020000,0.000000e+00,0.000000,3.459000e+08,0.0
9,5.397251e+07,0.000000,2.024272e+07,0.021137,2.879840e+07,0.000000,1.909032e+06,0.001173,4.172854e+08,0.411211,3.459000e+08,0.0


In [101]:
print(np.mean(outcomes["A.2 Total Costs"]))
print(np.mean(outcomes["A.1_Expected Number of Deaths"].iloc[0:20].mean()))
print(np.mean(outcomes["A.2_Expected Number of Deaths"].iloc[0:20].mean()))
print(np.mean(outcomes["A.3_Expected Number of Deaths"].iloc[0:20].mean()))
print(np.mean(outcomes["A.4_Expected Number of Deaths"].iloc[0:20].mean()))
print(np.mean(outcomes["A.5_Expected Number of Deaths"].iloc[0:20].mean()))
print(np.mean(outcomes["A.1_Expected Number of Deaths"].iloc[20:40].mean()))
print(np.mean(outcomes["A.2_Expected Number of Deaths"].iloc[20:40].mean()))
print(np.mean(outcomes["A.3_Expected Number of Deaths"].iloc[20:40].mean()))
print(np.mean(outcomes["A.4_Expected Number of Deaths"].iloc[20:40].mean()))
print(np.mean(outcomes["A.5_Expected Number of Deaths"].iloc[20:40].mean()))

84436408.65591046
0.0
0.07573830475411861
0.0011497821473481246
0.019820750144960868
0.1326741663628996
0.6701737431750037
0.0747519637416387
0.4134972364571315
0.0006300414427985935
0.011470645203987752


In [102]:
experiments = pd.DataFrame(experiments)
experiments

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,A.4_Bmax,A.4_Brate,A.4_pfail,A.5_Bmax,A.5_Brate,A.5_pfail,discount rate 0,discount rate 1,discount rate 2,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,3_RfR 1,3_RfR 2,4_RfR 0,4_RfR 1,4_RfR 2,EWS_DaysToThreat,A.1_DikeIncrease 0,A.1_DikeIncrease 1,A.1_DikeIncrease 2,A.2_DikeIncrease 0,A.2_DikeIncrease 1,A.2_DikeIncrease 2,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,scenario,policy,model
0,132,136.238000,1.5,0.188658,140.823628,1.0,0.464103,230.521316,1.5,0.465768,157.192930,10.0,0.048209,190.287273,1.0,0.679287,3.5,1.5,1.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,260,policy 1,dikesnet
1,56,230.742429,1.0,0.524480,102.779533,10.0,0.666173,144.628112,1.0,0.705111,183.930149,1.5,0.501662,221.204136,10.0,0.836614,3.5,2.5,4.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,261,policy 1,dikesnet
2,115,187.354849,1.5,0.145206,262.499172,1.5,0.581927,271.194757,1.5,0.432516,161.373010,1.0,0.998842,76.880464,10.0,0.770534,1.5,2.5,4.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,262,policy 1,dikesnet
3,50,75.829439,1.5,0.074273,244.838315,1.0,0.134901,256.832355,10.0,0.993096,79.012376,10.0,0.344402,255.205621,1.0,0.443189,3.5,2.5,4.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,263,policy 1,dikesnet
4,44,317.786513,1.0,0.296274,119.980888,1.5,0.752007,109.859912,10.0,0.035812,276.471678,1.0,0.615749,161.990763,1.5,0.935708,1.5,1.5,2.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,264,policy 1,dikesnet
5,6,80.876450,1.0,0.811885,83.196476,10.0,0.395835,44.343626,1.5,0.212533,204.890557,1.0,0.488140,87.803580,1.5,0.186463,2.5,3.5,2.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,265,policy 1,dikesnet
6,20,281.209482,10.0,0.023150,66.164573,1.5,0.879032,173.769898,1.0,0.842178,317.893432,1.5,0.590446,286.070077,1.5,0.379155,3.5,1.5,4.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,266,policy 1,dikesnet
7,109,112.512348,1.5,0.986003,154.865380,10.0,0.037369,178.194845,10.0,0.336718,51.855748,1.0,0.692401,279.880535,1.0,0.128544,2.5,3.5,1.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,267,policy 1,dikesnet
8,27,266.639767,10.0,0.225817,215.032889,10.0,0.095825,133.942106,1.0,0.359095,235.454588,1.0,0.233851,100.542020,10.0,0.867902,4.5,2.5,3.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,268,policy 1,dikesnet
9,16,299.901637,1.5,0.608114,331.946871,1.0,0.500700,119.497379,10.0,0.948047,246.170276,1.5,0.819841,186.637523,10.0,0.207821,2.5,4.5,2.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,269,policy 1,dikesnet
